In [ ]:
import itertools
from math import *

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from utils.colormap import truncate_colormap
from utils import itertools_recipes

from ml.gradient_descent import GradientDescentOptimizer

In [ ]:
def randomDet1Matrix():
    A = np.random.normal(size=(2, 2))
    A /= sqrt(abs(np.linalg.det(A)))
    return A

reds_m = 80
blues_m = 60

radiuses = np.random.normal(10, 1, size=reds_m) ** 2
angles = np.random.uniform(low=0.0, high=2 * pi, size=reds_m)
A = randomDet1Matrix()  # Random tilt / rotation
b = np.random.normal(0, 40, size=(2, 1)) # Random shift
reds = A @ np.array([radiuses * np.cos(angles), radiuses * np.sin(angles)]) + b
blues = A @ np.random.multivariate_normal([0, 0], [[700, 0], [0, 700]], blues_m).T + b

In [ ]:
M = reds_m + blues_m

In [ ]:
plt.scatter(*reds/150, c='r',  linewidths=0.5, edgecolors='black')
plt.scatter(*blues/150, c='b',  linewidths=0.5, edgecolors='black');

In [ ]:
unbiased_xs = np.c_[reds, blues] / 150 # Normalize by hand
products = np.prod(unbiased_xs, axis=0, keepdims=True)
squares = unbiased_xs ** 2
XS = np.r_[np.ones((1, unbiased_xs.shape[1])), unbiased_xs, products, squares]
YS = np.r_[np.zeros(reds.shape[1]), np.ones(blues.shape[1])]

In [ ]:
h = lambda θ: lambda x: 1.0 / (1.0 + exp(-np.dot(θ, x)))

In [ ]:
# red  => y=0
# blue => y=1

def J(θ):
    h_θ = h(θ)
    totalSum = sum((log(1 - h_θ(x)) if y == 0 else log(h_θ(x))) for x, y in zip(XS.T, YS))
    return -totalSum / M

In [ ]:
def dJ(θ):
    h_θ = h(θ)
    return 1 / M * sum(x * (h_θ(x) - y) for x, y in zip(XS.T, YS))

In [ ]:
optimizer = GradientDescentOptimizer(J=J, δJ=dJ, α=0.7, start_θ=[0, 0, 0, 0, 0, 0])

In [ ]:
θ_opt = optimizer.nth(100)

In [ ]:
-θ_opt

In [ ]:
xmin = -3
xmax = 10
ymin = -3
ymax = 6

def plotHypothesis(θ):
    z_func = np.vectorize(lambda x1, x2: h(θ)(np.array([1, x1, x2, x1 * x2, x1 ** 2, x2 ** 2])))
    Z = z_func(np.linspace(xmin, xmax, 50)[:, None], np.linspace(ymin, ymax, 50))

    plt.figure(figsize=(15,6))
    plt.imshow(
        Z.T,
        extent=(xmin, xmax, ymin, ymax),
        interpolation='none',
        cmap=truncate_colormap("PuOr", 0.3, 0.7),
        origin='lower'
    )
    plt.colorbar()
    plotData()
    plt.ylim(ymin, ymax)
    plt.xlim(xmin, xmax)

In [ ]:
plotHypothesis(θ_opt)
plt.scatter(*reds/150, c='r',  linewidths=0.5, edgecolors='black')
plt.scatter(*blues/150, c='b',  linewidths=0.5, edgecolors='black');